In [1]:
sc

In [6]:
lines = sc.textFile('input/file1')

In [7]:
lines.collect()

[u'Hello World Bye World']

In [9]:
words= lines.flatMap(lambda l: l.split(" "))

In [10]:
words.count()

4

In [17]:
rdd = sc.textFile("file:///home/vagrant/input/*")

In [18]:
counts = rdd.flatMap(lambda line: line.split(" ")) \
...              .map(lambda word: (word, 1)) \
...              .reduceByKey(lambda a, b: a + b)

In [19]:
counts.collect()

[(u'Goodbye', 1), (u'World', 2), (u'Bye', 1), (u'Hello', 2), (u'Hadoop', 2)]

In [20]:
purachaseRdd= sc.textFile('file:///home/vagrant/input/purchases.txt')

In [22]:
purachaseRdd.collect()

[u'2012-07-20 09:59:00,Corpus Christi,CDs,327.91,Cash',
 u'2012-03-11 17:29:00,Durham,Books,115.09,Discover',
 u'2012-07-31 11:43:00,Rochester,Toys,332.07,MasterCard',
 u'2012-06-18 14:47:00,Garland,Computers,31.99,Visa',
 u'2012-03-27 11:40:00,Tulsa,CDs,452.18,Discover',
 u'2012-05-31 10:57:00,Pittsburgh,Garden,492.25,Amex',
 u'2012-08-22 14:35:00,Richmond,Consumer Electronics,346,Amex',
 u'2012-09-23 16:45:00,Scottsdale,CDs,21.58,Cash',
 u'2012-10-17 11:29:00,Baton Rouge,Computers,226.26,Cash',
 u"2012-07-03 11:05:00,Virginia Beach,Women's Clothing,23.47,Cash",
 u"2012-08-18 09:31:00,Norfolk,Men's Clothing,379.33,Cash",
 u'2012-01-20 14:07:00,Wichita,Video Games,392.43,Cash',
 u'2012-11-27 12:16:00,Saint Paul,Books,281.64,Discover',
 u'2012-03-15 09:40:00,St. Petersburg,CDs,9.86,Visa',
 u'2012-01-02 10:39:00,Minneapolis,Baby,116.67,Amex',
 u'2012-09-30 09:13:00,El Paso,Health and Beauty,487.24,MasterCard',
 u'2012-09-20 13:52:00,Los Angeles,Health and Beauty,139.15,Discover',
 u'2012

In [23]:
transactions = purachaseRdd.map(lambda l: l.split(","))

In [28]:
from pyspark.sql import SQLContext, Row
sqlContext = SQLContext(sc)

In [33]:
transaction = transactions.map(lambda p: Row(timestamp =p[0], location =p[1], category =p[2],price =float(p[3]), card = p[4]))

In [34]:
schemaTransaction = sqlContext.createDataFrame(transaction)

In [35]:
schemaTransaction.registerTempTable("purchases")

In [39]:
avpPrice = sqlContext.sql("SELECT AVG(price) FROM purchases WHERE card='MasterCard'")

In [40]:
avpPrice

DataFrame[c0: double]

In [41]:
print(avpPrice)

DataFrame[c0: double]


In [45]:
avpPrice.c0

Column<c0>

In [46]:
avpPrice.show()

c0                
275.06773195876286


In [67]:
sale = sqlContext.sql("SELECT timestamp, SUM(price) AS total_sales FROM purchases GROUP BY timestamp SORT BY total_sales DESC LIMIT 1")

In [63]:
sale.show()

timestamp           total_sales
2012-02-10 10:40:00 439.3      


In [68]:
minValue = sqlContext.sql("SELECT MIN(price) FROM purchases WHERE category='Computers'")

In [69]:
minValue.show()

c0  
0.38


In [70]:
cnt = sqlContext.sql("SELECT COUNT(DISTINCT category) FROM purchases")

In [71]:
cnt.show()

c0
18


In [73]:
location = sqlContext.sql("SELECT location, SUM(price) AS store_sales FROM purchases GROUP BY location ORDER BY store_sales LIMIT 1")

In [74]:
location.show()

location store_sales
Plano    784.96     


In [76]:
transaction.take(5)

[Row(card=u'Cash', category=u'CDs', location=u'Corpus Christi', price=327.91, timestamp=u'2012-07-20 09:59:00'),
 Row(card=u'Discover', category=u'Books', location=u'Durham', price=115.09, timestamp=u'2012-03-11 17:29:00'),
 Row(card=u'MasterCard', category=u'Toys', location=u'Rochester', price=332.07, timestamp=u'2012-07-31 11:43:00'),
 Row(card=u'Visa', category=u'Computers', location=u'Garland', price=31.99, timestamp=u'2012-06-18 14:47:00'),
 Row(card=u'Discover', category=u'CDs', location=u'Tulsa', price=452.18, timestamp=u'2012-03-27 11:40:00')]

In [82]:
transactionDF = transaction.toDF(['card', 'category', 'location', 'price', 'timestamp'])

In [83]:
transactionDF.show()

card       category             location       price  timestamp          
Cash       CDs                  Corpus Christi 327.91 2012-07-20 09:59:00
Discover   Books                Durham         115.09 2012-03-11 17:29:00
MasterCard Toys                 Rochester      332.07 2012-07-31 11:43:00
Visa       Computers            Garland        31.99  2012-06-18 14:47:00
Discover   CDs                  Tulsa          452.18 2012-03-27 11:40:00
Amex       Garden               Pittsburgh     492.25 2012-05-31 10:57:00
Amex       Consumer Electronics Richmond       346.0  2012-08-22 14:35:00
Cash       CDs                  Scottsdale     21.58  2012-09-23 16:45:00
Cash       Computers            Baton Rouge    226.26 2012-10-17 11:29:00
Cash       Women's Clothing     Virginia Beach 23.47  2012-07-03 11:05:00
Cash       Men's Clothing       Norfolk        379.33 2012-08-18 09:31:00
Cash       Video Games          Wichita        392.43 2012-01-20 14:07:00
Discover   Books                Saint 

In [84]:
from pyspark.sql.functions import *

In [85]:
transactionDF.filter(transactionDF.card == 'MasterCard').agg(avg(col('price'))).show()

AVG(price)        
275.06773195876286
